<a href="https://colab.research.google.com/github/Haebuk/Python_Machine_Learning/blob/master/logfile_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/eliasdabbas/webserver-log-file-analysis

In [ ]:
import pandas as pd
import re
import os
import time
from tqdm import tqdm

for path, dir, filenames in os.walk('/content/drive/MyDrive/input/access.log'):
    for filename in filenames:
        logfile = os.path.join(path, filename)
        print(logfile)

/content/drive/MyDrive/input/access.log/access.log


- `os.listdir(dirname)`: `dirname`에 위치한 파일 목록 열거
- `os.path.join(dirname, filename)`: `dirname`과 `filename` 경로를 이어줌
- `os.path.splitext`: 파일명을 파일이름과 확장자명 두가지로 나눠줌

In [ ]:
!ls -lsSh /content/drive/MyDrive/input/access.log/access.log

3.3G -rw------- 1 root root 3.3G Jul 16 01:08 /content/drive/MyDrive/input/access.log/access.log


In [ ]:
def search(dirname):
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        ext = os.path.splitext(full_filename)[-1]
        if ext == '.csv':
            print(full_filename)

search('/content/drive/MyDrive/input/')

('/content/drive/MyDrive/input/stage1_sample_submission', '.csv')
/content/drive/MyDrive/input/stage1_sample_submission.csv
('/content/drive/MyDrive/input/stage1_solution', '.csv')
/content/drive/MyDrive/input/stage1_solution.csv
('/content/drive/MyDrive/input/stage1_train_labels', '.csv')
/content/drive/MyDrive/input/stage1_train_labels.csv
('/content/drive/MyDrive/input/stage2_sample_submission_final', '.csv')
/content/drive/MyDrive/input/stage2_sample_submission_final.csv
('/content/drive/MyDrive/input/stage2_test_final', '')
('/content/drive/MyDrive/input/stage1_train', '')
('/content/drive/MyDrive/input/stage1_test', '')
('/content/drive/MyDrive/input/mnist', '')
('/content/drive/MyDrive/input/hymenoptera_data', '')
('/content/drive/MyDrive/input/intel', '')
('/content/drive/MyDrive/input/.ipynb_checkpoints', '')
('/content/drive/MyDrive/input/programmers', '')
('/content/drive/MyDrive/input/dataminingTP', '')
('/content/drive/MyDrive/input/goodbooks10k', '')
('/content/drive/MyDr

`os.walk(input)`
- path: input
- dir: 폴더 리스트
- files: 파일 리스트

In [ ]:
with open(logfile) as log:
    for line in log.readlines():
        match = re.match(r'(.+) - - \[(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2}) \+\d{4}\] \"')

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"

40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/1.1" 200 16

In [ ]:
regex = '^(?P<client>\S+) - - \[(?P<datetime>[^*]+)\] "(?P<method>[A-Z]+) (?P<request>\S+) HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+) "(?P<url>\S+)" "(?P<useragent>[^"]*)"'
columns = ['client', 'datetime', 'method', 'request', 'status', 'size', 'url', 'useragent']

In [ ]:
%mkdir parquet_dir

In [ ]:
def logs_to_df(logfile, output_dir, errors_file):
    with open(logfile) as source_file:
        linenumber = 0
        parsed_lines = []
        for line in tqdm(source_file):
            try:
                log_line = re.findall(regex, line)[0]
                if linenumber % 50000 == 0:
                    print(log_line)
                parsed_lines.append(log_line)
            except Exception as e:
                with open(errors_file, 'at') as errfile:
                    print((line, str(e)), file=errfile)
                continue
            linenumber += 1
            if linenumber % 250000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear()
        else:
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
            parsed_lines.clear()


In [ ]:
%time logs_to_df(logfile=logfile, output_dir='parquet_dir', errors_file='errors.txt')

24494it [00:00, 121708.93it/s]

('54.36.149.41', '22/Jan/2019:03:56:14 +0330', 'GET', '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53', '200', '30577', '-', 'Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)')


70183it [00:00, 116789.35it/s]

('40.77.167.15', '22/Jan/2019:07:00:04 +0330', 'GET', '/filter/b67,b7,p62', '200', '30402', '-', 'Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)')


114006it [00:01, 111689.04it/s]

...

('5.126.27.221', '26/Jan/2019:19:39:27 +0330', 'GET', '/image/1176/mainSlideMobile', '200', '106246', 'https://www.zanbil.ir/m/browse/home-appliances/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C', 'Mozilla/5.0 (Linux; Android 7.0; LG-M400 Build/NRD90U; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Mobile Safari/537.36')


10317811it [02:20, 75794.42it/s]

('176.65.180.143', '26/Jan/2019:19:59:05 +0330', 'GET', '/image/60799/productModel/100x100', '200', '2965', 'https://www.zanbil.ir/filter/b854,p11671', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0')


10365152it [02:21, 73460.87it/s]


('91.99.26.60', '26/Jan/2019:20:21:25 +0330', 'GET', '/image/35/brand', '200', '2264', 'https://www.zanbil.ir/', 'Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1')
CPU times: user 2min 5s, sys: 8.29 s, total: 2min 14s
Wall time: 2min 21s


In [ ]:
!wc errors.txt

   314   7867 236189 errors.txt


In [ ]:
%time logs_df = pd.read_parquet('parquet_dir/')

CPU times: user 12.9 s, sys: 8.43 s, total: 21.3 s
Wall time: 15.5 s


In [ ]:
!du -sh parquet_dir/

257M	parquet_dir/


In [ ]:
logs_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364838 entries, 0 to 10364837
Data columns (total 8 columns):
 #   Column     Non-Null Count     Dtype 
---  ------     --------------     ----- 
 0   client     10364838 non-null  object
 1   datetime   10364838 non-null  object
 2   method     10364838 non-null  object
 3   request    10364838 non-null  object
 4   status     10364838 non-null  object
 5   size       10364838 non-null  object
 6   url        10364838 non-null  object
 7   useragent  10364838 non-null  object
dtypes: object(8)
memory usage: 632.6+ MB


In [ ]:
%rm -r parquet_dir/

In [ ]:
logs_df['client'] = logs_df['client'].astype('category')
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], format='%d/%b/%Y:%H:%M:%S %z')
logs_df['method'] = logs_df['method'].astype('category')
logs_df['status'] = logs_df['size'].astype('int32')
logs_df['url'] = logs_df['url'].astype('category')
logs_df['useragent'] = logs_df['useragent'].astype('category')

In [ ]:
logs_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364838 entries, 0 to 10364837
Data columns (total 8 columns):
 #   Column     Non-Null Count     Dtype                                
---  ------     --------------     -----                                
 0   client     10364838 non-null  category                             
 1   datetime   10364838 non-null  datetime64[ns, pytz.FixedOffset(210)]
 2   method     10364838 non-null  category                             
 3   request    10364838 non-null  object                               
 4   status     10364838 non-null  int32                                
 5   size       10364838 non-null  object                               
 6   url        10364838 non-null  category                             
 7   useragent  10364838 non-null  category                             
dtypes: category(4), datetime64[ns, pytz.FixedOffset(210)](1), int32(1), object(2)
memory usage: 404.7+ MB


In [ ]:
%time logs_df.to_parquet('logs_df.parquet')

CPU times: user 6.29 s, sys: 1.48 s, total: 7.77 s
Wall time: 7.5 s


In [ ]:
!ls -lshS

total 277M
277M -rw-r--r-- 1 root root 277M Jul 16 04:43 logs_df.parquet
236K -rw-r--r-- 1 root root 231K Jul 16 04:26 errors.txt
4.0K drwx------ 6 root root 4.0K Jul 16 01:11 drive
4.0K drwxr-xr-x 1 root root 4.0K Jul  1 13:42 sample_data


In [ ]:
%time logs_df = pd.read_parquet('logs_df.parquet')

CPU times: user 7.8 s, sys: 1.86 s, total: 9.65 s
Wall time: 7.37 s


In [ ]:
logs_df.shape

(10364838, 8)

In [ ]:
logs_df

,client,datetime,method,request,status,size,url,useragent
0,37.152.163.59,2019-01-22 12:38:27+03:30,GET,/static/images/loading.gif,7370,7370,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
1,77.245.233.52,2019-01-22 12:38:27+03:30,GET,/image/11082/productType/240x180,12458,12458,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
2,37.27.128.139,2019-01-22 12:38:27+03:30,GET,/browse/Tablet-Arm-Chair/%D8%B5%D9%86%D8%AF%D9...,30604,30604,https://www.zanbil.ir/browse/Classroom-Furnitu...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...
3,77.245.233.52,2019-01-22 12:38:27+03:30,GET,/image/851/mainSlide,89859,89859,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
4,77.245.233.52,2019-01-22 12:38:27+03:30,GET,/image/848/mainSlide,93168,93168,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
...,...,...,...,...,...,...,...,...
10364833,86.104.110.254,2019-01-26 16:01:31+03:30,GET,/image/64832/productModel/200x200,8667,8667,https://www.zanbil.ir/m/browse/tv/%D8%AA%D9%84...,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like M...
10364834,65.49.68.192,2019-01-26 16:01:31+03:30,GET,/image/64913/productModel/150x150,4395,4395,https://www.zanbil.ir/browse/audio-and-video-e...,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...
10364835,65.49.68.192,2019-01-26 16:01:31+03:30,GET,/image/64802/productModel/150x150,4490,4490,https://www.zanbil.ir/browse/audio-and-video-e...,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...
10364836,46.209.107.210,2019-01-26 16:01:31+03:30,GET,/image/%7B%7BbasketItem.id%7D%7D?type=productM...,5,5,https://www.zanbil.ir/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...


In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(k)

In [ ]:
%time kmeans.fit(logs_df[['size']])

CPU times: user 1min 13s, sys: 2.48 s, total: 1min 16s
Wall time: 1min 15s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
sorted(kmeans.cluster_centers_.round(0).flatten())

[3576.0, 27717.0, 73835.0, 191130.0, 549767.0]

In [ ]:
cluster_df = logs_df.groupby(kmeans.labels_)['size'].describe().sort_values('mean').reset_index(drop=True)

KeyError: ignored

In [ ]:
logs_df.groupby(kmeans.labels_)['size'].describe()

,count,unique,top,freq
0,1834988,31211,28536,113757
1,8021255,14872,0,770605
2,55039,7791,185207,30572
3,442381,8797,62894,55599
4,11175,7035,708098,222


In [ ]:
import numpy as np
a = [[2,1], [1,2]]
b = [[2,1], [2,2]]
np.dot(a,b)

array([[6, 4],
       [6, 5]])

In [ ]:
for line in open(logfile):
    print(line.split()[0])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5.121.43.23
66.249.66.91
180.163.220.3
66.249.66.194
66.249.66.91
5.121.43.23
172.20.2.174
5.121.43.23
66.249.66.91
66.249.66.194
66.249.66.91
5.121.43.23
66.249.66.194
66.249.66.194
66.249.66.194
2.186.65.58
5.115.54.117
66.249.66.194
207.46.13.136
207.46.13.136
180.163.220.3
180.163.220.4
180.163.220.3
207.46.13.136
54.36.148.232
66.249.66.194
180.163.220.68
207.46.13.136
66.249.66.91
180.163.220.4
5.121.43.23
54.36.148.129
66.249.66.194
66.249.66.194
66.249.66.194
207.46.13.136
5.121.43.23
66.249.66.194
207.46.13.136
54.36.148.232
66.249.66.194
66.249.66.91
66.249.66.91
66.249.66.194
66.249.66.194
51.15.15.54
66.249.66.91
66.249.66.194
66.249.66.91
66.249.66.194
5.200.112.161
66.249.66.194
5.121.43.23
66.249.66.194
66.249.66.91
66.249.66.194
66.249.66.194
54.36.149.50
66.249.66.194
5.160.157.20
5.160.157.20
66.249.66.91
17.58.102.43
83.120.223.38
5.236.128.80
66.249.66.194
17.58.102.43
31.58.190.45
5.124.197.62
66.249.66.91
66.249.66.91
66.249.66.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
46.4.203.149
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
2.190.156.108
5.211.32.40
5.62.222.160
5.62.222.160
5.62.222.160
5.62.222.160
5.62.222.160
54.36.148.29
5.62.222.160
66.249.66.194
207.46.13.136
5.113.40.47
157.55.39.245
5.62.222.160
2.190.156.108
5.62.222.160
46.4.203.149
66.249.66.194
157.55.39.245
54.36.148.105
5.62.222.160
5.113.40.47
5.113.40.47
66.249.66.91
66.249.66.91
5.211.32.40
5.62.222.160
172.20.2.174
66.249.66.194
54.36.148.132
54.36.148.74
157.55.39.245
5.62.222.160
66.249.66.194
40.77.195.83
66.249.66.194
157.55.39.245
66.249.66.194
66.249.66.91
66.249.66.91
66.249.66.194
204.18.201.158
188.229.71.67
5.160.157.20
5.160.157.20
5.113.40.47
66.249.66.194
54.36.148.167
5.62.222.160
5.62.222.160


KeyboardInterrupt: ignored